<a href="https://colab.research.google.com/github/finardi/tutos/blob/master/LlamaIndex_Local_Model_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LlamaIndex - Private Setup

Using GPT4ALL and our HuggingFace embeddings, we will injest [Chapter 3 of the recent IPCC Climate Report](https://www.ipcc.ch/report/ar6/wg2/chapter/chapter-3/), which covers oceans and coastal ecosystems. Using llama-index, this PDF is injested and vectorized, and questions can be answered about anything from this 172 paged report.

Climate reports are long and tedious to read, so this demo will also help give some insight to the latest findings from the IPCC!

Inspired by the recent popularity of [PrivateGPT](https://github.com/imartinez/privateGPT), this notebook will walk you through a llama-index setup that uses entirely local models. In this notebook, we use GPT4ALL and huggingface embeddings, which should run decently well on CPU alone. If you had more resources, we also provide some links further down for setting up any LLM from huggingface and running on GPU.

## Dependencies Setup

### Setup Line Wrapping

In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

### Download gpt4all model

In [2]:
!wget https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin

--2023-05-15 19:28:26--  https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin
Resolving gpt4all.io (gpt4all.io)... 104.26.0.159, 104.26.1.159, 172.67.71.169, ...
Connecting to gpt4all.io (gpt4all.io)|104.26.0.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3785248281 (3.5G)
Saving to: ‘ggml-gpt4all-j-v1.3-groovy.bin’

ggml-gpt4all-j-v1.3 100%[===================>]   3.52G  64.8MB/s    in 61s     

2023-05-15 19:29:28 (58.8 MB/s) - ‘ggml-gpt4all-j-v1.3-groovy.bin’ saved [3785248281/3785248281]



### Download 2023 IPPC Climate Report - Chapter 3 on Oceans and Coastal Ecosystems (172 Pages)

In [3]:
!wget https://www.ipcc.ch/report/ar6/wg2/downloads/report/IPCC_AR6_WGII_Chapter03.pdf

--2023-05-15 19:29:28--  https://www.ipcc.ch/report/ar6/wg2/downloads/report/IPCC_AR6_WGII_Chapter03.pdf
Resolving www.ipcc.ch (www.ipcc.ch)... 172.67.16.59, 104.20.24.161, 104.20.23.161, ...
Connecting to www.ipcc.ch (www.ipcc.ch)|172.67.16.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21752444 (21M) [application/pdf]
Saving to: ‘IPCC_AR6_WGII_Chapter03.pdf’

IPCC_AR6_WGII_Chapt 100%[===================>]  20.74M  --.-KB/s    in 0.09s   

2023-05-15 19:29:28 (236 MB/s) - ‘IPCC_AR6_WGII_Chapter03.pdf’ saved [21752444/21752444]



### Download extra packages

In [4]:
!pip install pymupdf pygpt4all llama-index sentence_transformers accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 73.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.3/273.3 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.0/246.0 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 834.2/834.2 kB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 109.3 MB/s eta 0:00:00
     ━━━━━━━━

## Documents setup

Here, we use PyMuPDFReader to quickly load all 172 pages of the climate report PDF. The `metadata=True` option will automatically set some helpful information like page numbers and filename, to help us keep track of sources.

In [5]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.llms import GPT4All
from llama_index.node_parser.simple import SimpleNodeParser
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter
from llama_index import (
    GPTVectorStoreIndex, 
    LangchainEmbedding, 
    LLMPredictor, 
    ServiceContext, 
    StorageContext, 
    download_loader,
    PromptHelper
)

In [6]:
PyMuPDFReader = download_loader("PyMuPDFReader")

In [7]:
documents = PyMuPDFReader().load(file_path='./IPCC_AR6_WGII_Chapter03.pdf', metadata=True)

# ensure document texts are not bytes objects
for doc in documents:
    doc.text = doc.text.decode()

In [8]:
# print a document to test. Each document is a single page from the pdf, with appropriate metadata
documents[10]

Document(text='3\n389\nOceans and Coastal Ecosystems and Their Services  \nChapter 3\noverlapping climate-induced drivers and non-climate drivers confound \nimplementation and assessment of the success of marine adaptation, \nrevealing the complexity of attempting to maintain marine ecosystems \nand services through adaptation. SROCC assessed with high confidence \nthat while the benefits of many locally implemented adaptations exceed \ntheir disadvantages, others are marginally effective and have large \ndisadvantages, and overall, adaptation has a limited ability to reduce the \nprobable risks from climate change, being at best a temporary solution \n(Bindoff et\xa0al., 2019a). SROCC also concluded that a portfolio of many \ndifferent types of adaptation actions, effective and inclusive governance, \nand mitigation must be combined for successful adaptation (Bindoff \net\xa0al., 2019a). The portfolio of adaptation measures has now been defined \n(Section\xa03.6.2), and individual and

## CPU Llama Index
The GPT4ALL setup follows the instructions from [langchain](https://python.langchain.com/en/latest/modules/models/llms/integrations/gpt4all.html).

Then, the model is wrapped in the LLMPredictor class from llama-index. 

Keep in mind this current setup will run on CPU. If you have access to a GPU, you could also run any LLM from huggingface for improved speed and performance. More details available on huggingface LLMs and example notebooks [here](https://gpt-index.readthedocs.io/en/latest/how_to/customization/custom_llms.html#example-using-a-huggingface-llm).

Lastly, the embeddings are downloaded and run locally using huggingface. These will automatically run on GPU if you have CUDA installed, otherwise they will run on CPU.

In [9]:
local_llm_path = './ggml-gpt4all-j-v1.3-groovy.bin'
llm = GPT4All(model=local_llm_path, backend='gptj', streaming=True, n_ctx=1024)
llm_predictor = LLMPredictor(llm=llm)

In [10]:
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

Note that due to model limitations (and considerations for speed), these settings are non-standard, but should help ensure reasonable resource usage with GPT4ALL

In [11]:
prompt_helper = PromptHelper(max_input_size=1024, num_output=256, max_chunk_overlap=-1000)
service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor,
    embed_model=embed_model,
    prompt_helper=prompt_helper,
    node_parser=SimpleNodeParser(text_splitter=TokenTextSplitter(chunk_size=512, chunk_overlap=50))
)

In [12]:
# Quick sanity test that the LLM works (seems to only work if you run this here!)
print(llm._call("hello world"))

!
Hello! How can I assist you today?


### Create the Index

This step will break each document into nodes, and create an embedding vector for each node using our `embed_model`. This may take a several minutes if running on CPU (this is a large climate report)!

In [13]:
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

In [14]:
index.storage_context.persist(persist_dir="./storage")

#### (Optional) Load the Index if already saved

In [15]:
from llama_index import load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir="./storage")
index = load_index_from_storage(storage_context, service_context=service_context)

### Try Asking a question

Due to processing constraints, setting `similarity_top_k=1` is an ideal setting. Otherwise, responses will be quite slow due to the speed of CPU inference. Either way, expect to wait a minute or two before the response is streaming.

In [16]:
query_engine = index.as_query_engine(streaming=True, similarity_top_k=1, service_context=service_context)

In [ ]:
response_stream = query_engine.query("What are the main climate risks to our Oceans?")
response_stream.print_response_stream()


The main climate risks to our oceans are:

1. Sea level rise: Climate change is causing the Earth's oceans to warm and expand, leading to a rise in sea level. This can cause flooding, erosion, and coastal erosion, as well as impact on marine ecosystems and the coastal communities that rely on them.
2. Ocean acidification: Increased levels of carbon dioxide in the atmosphere are leading to an increase in the acidity of the oceans, which can have a negative impact on marine ecosystems and human health.
3. Coastal erosion: Climate change is causing sea levels to rise, leading to increased erosion of coastlines and coastal

The above response is hallucinated slightly, since all that was not availabe in the provided top-1 releveant node. In any case, it grounded the response and gave a generally OK response

## GPU Llama Index

As stated earlier, if you have a modest GPU available (at least 15GB of VRAM), you can speed things up considerably.

This next section will setup a new predictor from Huggingface, using the Writer/camel-5b-hf model (which is also conviently licensed for commericial use).

(If you are running in colab, switch to a GPU instance first!)

### LLM + Embed Model Setup

In [ ]:
# setup prompts - specific to Camel
from llama_index.prompts.prompts import SimpleInputPrompt

# This will wrap the default prompts that are internal to llama-index
# taken from https://huggingface.co/Writer/camel-5b-hf
query_wrapper_prompt = SimpleInputPrompt(
    "Below is an instruction that describes a task. "
    "Write a response that appropriately completes the request.\n\n"
    "### Instruction:\n{query_str}\n\n### Response:"
)

In [ ]:
import torch
from llama_index.llm_predictor import HuggingFaceLLMPredictor

# NOTE: the first run of this will download/cache the weights, ~20GB
hf_predictor = HuggingFaceLLMPredictor(
    max_input_size=2048, 
    max_new_tokens=256,
    temperature=0.25,
    do_sample=False,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="Writer/camel-5b-hf",
    model_name="Writer/camel-5b-hf",
    device_map="auto",
    tokenizer_kwargs={"max_length": 2048},
    model_kwargs={"torch_dtype": torch.bfloat16}
)

embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

service_context = ServiceContext.from_defaults(chunk_size_limit=512, llm_predictor=hf_predictor, embed_model=embed_model)

### Construct index using GPU

Since we are using a GPU now, the emebddings are much faster to generate! Plus, now we are working with larger chunks, which enables the embeddings to represent the text and enable better retrieval.

In [ ]:
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)
index.storage_context.persist(persist_dir="./storage")

#### (Optional) Load if already saved

In [ ]:
from llama_index import load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir="./storage")
index = load_index_from_storage(storage_context, service_context=service_context)

### Query using GPU

With a GPU, the response will begin streaming very quickly. Camel is an excellent model given it's modest size of 5B parameters!

In [ ]:
query_engine = index.as_query_engine(streaming=True, similarity_top_k=3, service_context=service_context)

In [ ]:
response_stream = query_engine.query("What are the main climate risks to our Oceans?")
response_stream.print_response_stream()

Compared to GPT4ALL, this response is more complete and accurate. You can inspect the source nodes below:

In [ ]:
print(response_stream.source_nodes)